Connected to lalor0 (Python 3.11.4)

In [1]:
from scipy import signal
import numpy as np
def get_segments(envelope:np.ndarray,fs,params=None):
    '''
    envelope: broadband envelope, assumed to be a 1D array
    function that uses smoothed envelope thresholding to get indices for non-silent segments
    in which match waves will look for stimuli
    params: {
    'filt_ord': sharpness of filter applied on smoothing envelope
    'filt_freqs': lowpass filter frequency (or list bandpass limits if bandpass desired)
    'min_sil': minimum silent period duration in seconds; 
    }

    returns
    -
    smooth_envelope: smoothed envelope normalized between 0 and 1
    segments: [n_segments x 2] array with first col containing indices of 
                                segment start and second col the end
    '''
    smooth_envelope=envelope.copy() #NOTE: useful during debugging but won't need once finished
    assert smooth_envelope.ndim==1, "this function assumes 1D array!"
    if params is None:
        params={
            'filt_ord':16,
            'filt_freqs':0.1,
            'min_sil': 3,
            'seg_padding': 2
        }
    if isinstance(params['filt_freqs'], list) and len(params['filt_freqs'])==2:
        _filt_type='bandpass'
    elif isinstance(params['filt_freqs'],float):
        _filt_type='low'
    else:
        raise NotImplementedError(f"params['filt_freqs'] should be float or int-> {type(params['filt_freqs'])}")
    # rectify since we looking for overall magnitude changes
    smooth_envelope[smooth_envelope<0]*=-1.0
    # smooth the envelope
    sos=signal.butter(params['filt_ord'],params['filt_freqs'],btype=_filt_type,output='sos',fs=fs)
    smooth_envelope=signal.sosfiltfilt(sos,smooth_envelope)
    # normalize between zero and 1
    smooth_envelope=(smooth_envelope-smooth_envelope.min())/(smooth_envelope.max()-smooth_envelope.min())
    
    assert np.all(smooth_envelope<=1) and np.all(smooth_envelope>=0), "envelope range should be between 0 and 1 here!"
    # get indices where smoothed envelope crosses half-median, should be 1 where envelope goes above .5 
    # the range and -1 where it goes back below
    loud_bits=(smooth_envelope>0.5*np.median(smooth_envelope)).astype(int)
    crossings=np.diff(loud_bits,prepend=0)
    # separate onsets from offsets, then pad 
    #TODO: add condition to check that amount padding via shift doesn't shift any onsets or offsets 
    # past start/end of recording array size
    # pad onsets by removing number of samples equal to seg_padding
    n_shift=int(params['seg_padding']*fs)
    onsets=np.concatenate((crossings[n_shift:]==1, np.zeros(n_shift)))
    offsets=np.concatenate((np.zeros(n_shift), crossings[:-n_shift]==-1))
    pause_durs=(np.argwhere(onsets[1:])-np.argwhere(offsets[:-1]))/fs #in seconds
    # remove excessively short pauses
    if np.any(pause_durs<params['min_sil']):
        rmv_indx=pause_durs<params['min_sil']
        onsets[np.argwhere(onsets[1:])[rmv_indx]]=0
        offsets[np.argwhere(offsets[:-1])[rmv_indx]]=0

    segments=np.hstack([np.argwhere(onsets),np.argwhere(offsets)])
    #TODO: check that segments is n x 2 array
    return segments, smooth_envelope